In [123]:
import os
import requests
import datetime

import numpy as np
import pandas as pd

from bs4 import BeautifulSoup

In [134]:
datestr = '3/2'




# datetime.datetime.strptime(datestr, "%m/%d")
current_date_str

'03/02'

In [45]:
class Conversions:
    
    def __init__(self):
        
        
        
        self.inits_teams: dict[str, str] = {
            'NY': 'New York Knicks',
            'LAL': 'Los Angeles Lakers',
            'MIA': 'Miami Heat',
            'UTA': 'Utah Jazz',
            'PHO': 'Phoenix Suns',
            'LAC': 'Los Angeles Clippers',
            # 'LAC': 'LA Clippers',
            'PHI': 'Philadelphia 76ers',
            'DAL': 'Dallas Mavericks',
            'DEN': 'Denver Nuggets',
            'BOS': 'Boston Celtics',
            'ATL': 'Atlanta Hawks',
            'CLE': 'Cleveland Cavaliers',
            'DET': 'Detroit Pistons',
            'TOR': 'Toronto Raptors',
            'CHA': 'Charlotte Hornets',
            'ORL': 'Orlando Magic',
            'MEM': 'Memphis Grizzlies',
            'SA': 'San Antonio Spurs',
            'MIL': 'Milwaukee Bucks',
            'IND': 'Indiana Pacers',
            'CHI': 'Chicago Bulls',
            'OKC': 'Oklahoma City Thunder',
            'GS': 'Golden State Warriors',
            'HOU': 'Houston Rockets',
            'BKN': 'Brooklyn Nets',
            'POR': 'Portland Trail Blazers',
            'NO': 'New Orleans Pelicans',
            'MIN': 'Minnesota Timberwolves',
            'SAC': 'Sacramento Kings',
            'WAS': 'Washington Wizards'
        }

        # Invert
        self.teams_inits: dict[str,str] = { val: key for key, val in self.inits_teams.items() }
        
    def team_name(self, team_str: str) -> str:
        return self.teams_inits[team_str]
    
    def team_initials(self, team_init_str: str) -> str:
        return self.inits_teams[team_init_str]

In [168]:
class PropsScraper:
    
    def __init__(self):
        self.convert = Conversions()
        self.directory_url: str = 'https://www.scoresandodds.com/nba/players'
        
        self.current_date_str = datetime.datetime.now().strftime("%m/%d")
    
#     Creates a dictionary containing the links to current and historical props
#     for every player in the NBA, organized by team
    def create_webpage_directory(self) -> dict[str, dict[str, str]]:
        
#         Load HTML into bs4
        soup = BeautifulSoup(
            requests.get(self.directory_url).text,
            'html.parser'
        )

#         Load each team data into dictionary, converting the full team name into initials as used in rest of data
        team_modules = {
            self.convert.team_name(team_html.find('h3').get_text()): team_html.find_all('div', class_='module-body')[0].find('ul')
            for team_html in soup.find_all('div', class_='module')
        }
        
        
#         Parse HTML data for each team to organize links in easily searchable manner
        teams_players_links: dict[str, dict[str, str]] = {
            
            team: {
                a_tag.get_text(): self.directory_url.replace(
                    '/nba/players',
                    a_tag['href']
                )
                for a_tag in module.find_all('a')
            }
            
            for team, module in team_modules.items()
            
        }
        
        return teams_players_links
    
    
    def scrape_player_props(self, name, url):
        
#         Load HTML
        soup = BeautifulSoup(
            requests.get(url).text, 
            'html.parser'
        )
        
        module = soup.find('div', class_="module-body scroll")
        
        if not len(module.find_all('span')):
            return dict()
        
#         Make sure current
        zerofill = lambda dp: f'0{dp}' if len(dp) == 1 else dp
        date_str = '/'.join([
            zerofill(dp) for dp in module.find_all('span')[2].get_text().split(' ')[1].split('/')
        ])
        
        if date_str != self.current_date_str:
            return dict()
        
        props_rows = module.find('table', class_='sticky').find('tbody').find_all('tr')
        
        # Steals, blocks, 3s are options but noisy, better to use season data for opponents
        targets = ('Points', 'Rebounds', 'Assists')
        
#         Form: Category Line Over Under
        target_rows = [row for row in props_rows if row.find('td').get_text() in targets]
    
        props = {
            'stat': list(),
            'line': list(),
            'over': list(),
            'under': list()
        }
    
        for rowtags in target_rows:
            vals = [val.get_text().lower() for val in rowtags.find_all('td')] # (Category, Line, Over, Under)
            props['stat'].append(vals[0])
            props['line'].append(vals[1])
            props['over'].append(vals[2])
            props['under'].append(vals[3])
        
        return props

In [ ]:
def testing():
    Props = PropsScraper()
    
    name: str = 'James Harden'
    url: str = 'https://www.scoresandodds.com/prop-bets/1037/james-harden'
    # bad_url: str = 'https://www.scoresandodds.com/prop-bets/1311802/louis-king'

    return Props.scrape_player_props(name, url)

In [ ]:
testing()